# Create HiRID Temporal External Validation Dataset (Experiment 2)

In [ ]:
#Import all necessary modules

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
#Connect to HiRID database

import psycopg2
from psycopg2 import Error

# Connect to HiRID
conn = psycopg2.connect(user="mimicuser",
                                  password="knowlabMIMIC",
                                  host="172.17.0.1",
                                  port="5433",
                                  database="HiRID")

# Cursor 
cur = conn.cursor()

### 1) Filtering for all instances with drug administration (Adrenaline/Noradrenaline)

HiRID documentation contains the variableids for all drug variables:<br>
https://hirid.intensivecare.ai/Structure-of-the-published-data-de050e11529140178dea1a1b925d558f

- **Adrenaline** (continuously administered): 1000649, 1000650
- **Noradrenaline** (continuously administered): 1000656, 1000657

In [ ]:
#Connect to pharma_records table
##filtering for only continuous Adrenaline/ Noradrenaline fields
##pharmaids for continuous Adrenaline/Noradrenaline fields found in HiRID documentation - https://hirid.intensivecare.ai/Structure-of-the-published-data-de050e11529140178dea1a1b925d558f

#import all records from HiRID Pharma table containing continuous Adrenaline/Noradrenaline
drugs_df = pd.read_sql_query("SELECT * FROM hirid.pharma_records \
                         WHERE pharmaid IN (1000649, 1000650, 1000656, 1000657) \
                         ORDER BY patientid, pharmaid, givenat", conn)


#categorise drugs_df 'givenat' column to closest 15min timepoint (needed for later analysis)
drugs_df['timepoint'] = drugs_df['givenat'].dt.round('15min') 

print(drugs_df.shape)
drugs_df.head()

##1,458,539 records
##NOTE: includes rows where given_dose=0

In [ ]:
#Filter for records containing Adrenaline/Noradrenaline readings with non-zero values 

drugs_nonzero = drugs_df[drugs_df.givendose != 0] #drop all rows where givendose=0.0

print(drugs_nonzero.shape)
drugs_nonzero

##1,313,057 records

In [ ]:
drugs_df.dtypes

In [ ]:
#if necessary - convert givenat to datetime format
drugs_df['givenat'] =  pd.to_datetime(drugs_df['givenat'], format='%Y-%m-%d %H:%M:%S.%f')
drugs_df.dtypes

In [ ]:
#show count of each drug field

drugs_df['pharmaid'].value_counts()

In [ ]:
#show all route values

drugs_df['route'].unique()

##only continuous administration - as required 

In [ ]:
#number of unique ICU admissions

pat_drugs = list(drugs_df['patientid'].unique())
len(pat_drugs)

#10,665 unique ICU admissions

In [ ]:
#Check number of unique instances (patientid + givenat)

agg = drugs_df.groupby(['patientid','givenat'])['pharmaid'].nunique()
agg = pd.DataFrame(data=agg, index=None)
agg = agg.reset_index()
print(agg.shape)
agg.head(10)

##1,430,607 unique instances 

In [ ]:
#Filter for 15min time intervals (per patientid) containing (non-zero) readings for Adrenaline/Noradrenaline

agg_drugs = drugs_nonzero.groupby([pd.Grouper(key='givenat', freq='15Min'), 'patientid']).pharmaid.unique()
agg_drugs = pd.DataFrame(data=agg_drugs)
agg_drugs = agg_drugs.reset_index()
agg_drugs = agg_drugs.sort_values(by=['patientid','givenat'], ascending=[True,True])

print(agg_drugs.shape)
agg_drugs.head(15)

#818,425 unique instances (with non-zero values for Adrenaline/Noradrenaline) 

In [ ]:
#check datatypes - givenat should be in datetime format

agg_drugs.dtypes

In [ ]:
#check unique ICU admissions

agg_drugs['patientid'].nunique()

##10,634 unique ICU admissions that have non-zero values for Adrenaline/Noradrenaline

### 2) Calculate drug rate for instances with Adrenaline/Noradrenaline drug readings (mg/hr)

In [ ]:
#Order drugs_df table by patientid, pharmaid, givenat

drugs_df.sort_values(by=['patientid','pharmaid','givenat'])

print(drugs_df.shape)
drugs_df.head()

##1,458,539 records

In [ ]:
#Confirm givenat is in datetime format

drugs_df.dtypes

In [ ]:
#Extract relevant columns from drugs_df

drugs_mod = drugs_df.copy(deep=True)
drugs_mod = drugs_mod[['patientid', 'pharmaid', 'givenat', 'timepoint', 'givendose','cumulativedose','doseunit']]

print(drugs_mod.shape)
drugs_mod.head()

##1,458,539 records

In [ ]:
#Check drug units

drugs_mod['doseunit'].unique()

##unit = micrograms

In [ ]:
#Get min_time for each (patientid, pharmaid, timepoint)
##min_time = minimum (earliest) drug administration time within given timepoint

temp = drugs_mod.groupby(['patientid','pharmaid', 'timepoint'])
min_time = temp.agg(min_time=('givenat', np.min))
min_time = min_time.reset_index()
min_time

In [ ]:
#Get cumulative dose for min_time per patientid in drugs_mod  

merge = pd.merge(min_time, drugs_mod, left_on=['patientid','pharmaid','timepoint','min_time'], right_on=['patientid','pharmaid','timepoint','givenat'])
min_cumval = merge[['patientid', 'pharmaid','timepoint','min_time','cumulativedose']]
min_cumval.rename({'cumulativedose': 'cumulativedose(min)'}, axis=1, inplace=True)
min_cumval

In [ ]:
#Get max_time for each (patientid, pharmaid, timepoint)
##max_time = maximum (latest) drug administration time within given timepoint

temp = drugs_mod.groupby(['patientid','pharmaid','timepoint'])
max_time = temp.agg(max_time=('givenat', np.max))
max_time = max_time.reset_index()
max_time

In [ ]:
#Get cumulative dose for max_time per patientid in drugs_mod  

merge = pd.merge(max_time, drugs_mod, left_on=['patientid','pharmaid','timepoint','max_time'], right_on=['patientid','pharmaid','timepoint','givenat'])
max_cumval = merge[['patientid', 'pharmaid','timepoint','max_time','cumulativedose']]
max_cumval.rename({'cumulativedose': 'cumulativedose(max)'}, axis=1, inplace=True)
max_cumval

In [ ]:
#Calculate drugs rate

drugs_rate = pd.merge(min_cumval, max_cumval, left_on=['patientid','pharmaid','timepoint'], right_on=['patientid','pharmaid','timepoint'])

drugs_rate['time_diff'] = drugs_rate['max_time']-drugs_rate['min_time']
drugs_rate['time_diff(mins)'] = (drugs_rate['time_diff'] / np.timedelta64(1, 'h'))*60
drugs_rate['cumulativedose_diff'] = drugs_rate['cumulativedose(max)']-drugs_rate['cumulativedose(min)']

drugs_rate['rate(microg/min)']= drugs_rate['cumulativedose_diff']/drugs_rate['time_diff(mins)']
drugs_rate

In [ ]:
#Covert rate units from microgrames/min to milligrams/hr

conversion = 0.060000071999942

drugs_rate['rate(millig/hr)']=drugs_rate['rate(microg/min)']*conversion
drugs_rate

In [ ]:
#Filter for acceptable rate values 

drugs_rate = drugs_rate[drugs_rate['rate(millig/hr)']<10]
drugs_rate = drugs_rate[drugs_rate['rate(millig/hr)']>0]

print(drugs_rate.shape)
drugs_rate

##272,932 records

In [ ]:
#Check min and max drug rate

max_rate = drugs_rate['rate(millig/hr)'].max()
min_rate = drugs_rate['rate(millig/hr)'].min()
print("Lowest drug rate:", min_rate)
print("Highest drug rate:", max_rate)

In [ ]:
#Rename pharmaid to drug name 

drugs_rate['pharmaid'] = drugs_rate['pharmaid'].replace({ 1000649 : "Adrenaline", 1000650 : "Adrenaline", 
                                                         1000656: "Noradrenaline", 1000657 : "Noradrenaline"})
drugs_rate.head()

In [ ]:
#Extract relevant columns from drugs_rate

drugs_rate = drugs_rate[['patientid','pharmaid','timepoint','rate(millig/hr)']]

print(drugs_rate.shape)
drugs_rate.head()

In [ ]:
#Reformat drugs_rate DF to structure similar to training data 

drugs_rate = drugs_rate.groupby(['patientid', 'timepoint', 'pharmaid'])['rate(millig/hr)'].sum().unstack('pharmaid')

drugs_rate = drugs_rate.reset_index()
drugs_rate = drugs_rate.rename_axis(None, axis=1)

drugs_rate['Adrenaline'] = round(drugs_rate['Adrenaline'],2)
drugs_rate['Noradrenaline'] = round(drugs_rate['Noradrenaline'],2)

drugs_rate.to_csv('drugs_rate.csv')

print(drugs_rate.shape)
drugs_rate

#268,461 records

### 3) Find instances with physiological parameter readings (FiO2/SpO2/HR/MAP)

HiRID documentation contains the variableids for all physiological parameters:<br>
https://hirid.intensivecare.ai/Structure-of-the-published-data-de050e11529140178dea1a1b925d558f

- **FiO2**: 2010
- **SpO2**: 4000, 8280
- **Heart Rate**: 200
- **Mean Arterial Pressure**: 110, 610

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Connect to observations_table_1

obs_tab1 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_1 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab1['row_idx']=obs_tab1.index #create new index column

obs_tab1.to_csv('obs_tab1.csv')

print(obs_tab1.shape)
obs_tab1.head()

#number of records = 19,492,758

In [ ]:
#Connect to observations_table_2

obs_tab2 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_2 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab2['row_idx']=obs_tab2.index #create new index column

obs_tab2.to_csv('obs_tab2.csv')

print(obs_tab2.shape)
obs_tab2.head()

#number of records = 19,373,029

In [ ]:
#Connect to observations_table_3

obs_tab3 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_3 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab3['row_idx']=obs_tab3.index #create new index column

obs_tab3.to_csv('obs_tab3.csv')

print(obs_tab3.shape)
obs_tab3.head()

#number of records = 19,922,456

In [ ]:
#Connect to observations_table_4

obs_tab4 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_4 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab4['row_idx']=obs_tab4.index #create new index column

obs_tab4.to_csv('obs_tab4.csv')

print(obs_tab4.shape)
obs_tab4.head()

#number of records = 19,576,004

In [ ]:
#Connect to observations_table_5

obs_tab5 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_5 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab5['row_idx']=obs_tab5.index #create new index column

obs_tab5.to_csv('obs_tab5.csv')

print(obs_tab5.shape)
obs_tab5.head()

#number of records = 19,475,848

In [ ]:
#Connect to observations_table_6

obs_tab6 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_6 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab6['row_idx']=obs_tab6.index #create new index column

obs_tab6.to_csv('obs_tab6.csv')

print(obs_tab6.shape)
obs_tab6.head()

#number of records = 19,612,923

In [ ]:
#Connect to observations_table_7

obs_tab7 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_7 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab7['row_idx']=obs_tab7.index #create new index column

obs_tab7.to_csv('obs_tab7.csv')

print(obs_tab7.shape)
obs_tab7.head()

## number of records = 19,818,593

In [ ]:
#Connect to observations_table_8

obs_tab8 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_8 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab8['row_idx']=obs_tab8.index #create new index column

obs_tab8.to_csv('obs_tab8.csv')

print(obs_tab8.shape)
obs_tab8.head()

#number of records = 19,449,724

In [ ]:
#Connect to observations_table_9

obs_tab9 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_9 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab9['row_idx']=obs_tab9.index #create new index column

obs_tab9.to_csv('obs_tab9.csv')

print(obs_tab9.shape)
obs_tab9.head()

#number of records = 19,922,926

In [ ]:
#Connect to observations_table_10

obs_tab10 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_10 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab10['row_idx']=obs_tab10.index #create new index column

obs_tab10.to_csv('obs_tab10.csv')

print(obs_tab10.shape)
obs_tab10.head()

#number of records = 18,538,580

In [ ]:
print('Number of records observation_table_1:', len(obs_tab1))
print('Number of records observation_table_2:', len(obs_tab2))
print('Number of records observation_table_3:', len(obs_tab3))
print('Number of records observation_table_4:', len(obs_tab4))
print('Number of records observation_table_5:', len(obs_tab5))
print('Number of records observation_table_6:', len(obs_tab6))
print('Number of records observation_table_7:', len(obs_tab7))
print('Number of records observation_table_8:', len(obs_tab8))
print('Number of records observation_table_9:', len(obs_tab9))
print('Number of records observation_table_10:', len(obs_tab10))

In [ ]:
#Import obs_tab1

obs_tab1= pd.read_csv("obs_tab1.csv")
obs_tab1= obs_tab1.drop(['Unnamed: 0'],axis=1)

obs_tab1['datetime'] =  pd.to_datetime(obs_tab1['datetime'], format='%Y-%m-%d %H:%M:%S')
obs_tab1['timepoint'] = obs_tab1['datetime'].dt.round('15min') #timepoint col needed in later manipulationn
print(obs_tab1.shape)
obs_tab1

##19,492,758 records

In [ ]:
#Import obs_tab2

obs_tab2= pd.read_csv("obs_tab2.csv")
obs_tab2= obs_tab2.drop(['Unnamed: 0'],axis=1)

obs_tab2['datetime'] =  pd.to_datetime(obs_tab2['datetime'], format='%Y-%m-%d %H:%M:%S')
obs_tab2['timepoint'] = obs_tab2['datetime'].dt.round('15min') #timepoint col needed in later manipulationn
print(obs_tab2.shape)
obs_tab2

##19,373,029 records

In [ ]:
#Import obs_tab3

obs_tab3= pd.read_csv("obs_tab3.csv")
obs_tab3= obs_tab3.drop(['Unnamed: 0'],axis=1)

obs_tab3['datetime'] =  pd.to_datetime(obs_tab3['datetime'], format='%Y-%m-%d %H:%M:%S')
obs_tab3['timepoint'] = obs_tab3['datetime'].dt.round('15min') #timepoint col needed in later manipulationn
print(obs_tab3.shape)
obs_tab3

##19,922,456 records

In [ ]:
#Import obs_tab4

obs_tab4= pd.read_csv("obs_tab4.csv")
obs_tab4= obs_tab4.drop(['Unnamed: 0'],axis=1)

obs_tab4['datetime'] =  pd.to_datetime(obs_tab4['datetime'], format='%Y-%m-%d %H:%M:%S')
obs_tab4['timepoint'] = obs_tab4['datetime'].dt.round('15min') #timepoint col needed in later manipulationn
print(obs_tab4.shape)
obs_tab4

##19,576,004 records

In [ ]:
#Import obs_tab5

obs_tab5= pd.read_csv("obs_tab5.csv")
obs_tab5= obs_tab5.drop(['Unnamed: 0'],axis=1)

obs_tab5['datetime'] =  pd.to_datetime(obs_tab5['datetime'], format='%Y-%m-%d %H:%M:%S')
obs_tab5['timepoint'] = obs_tab5['datetime'].dt.round('15min') #timepoint col needed in later manipulationn
print(obs_tab5.shape)
obs_tab5

##19,475,848 records

In [ ]:
#Import obs_tab6

obs_tab6= pd.read_csv("obs_tab6.csv")
obs_tab6= obs_tab6.drop(['Unnamed: 0'],axis=1)

obs_tab6['datetime'] =  pd.to_datetime(obs_tab6['datetime'], format='%Y-%m-%d %H:%M:%S')
obs_tab6['timepoint'] = obs_tab6['datetime'].dt.round('15min') #timepoint col needed in later manipulationn
print(obs_tab6.shape)
obs_tab6

##19,612,923 records

In [ ]:
#Import obs_tab7

obs_tab7= pd.read_csv("obs_tab7.csv")
obs_tab7= obs_tab7.drop(['Unnamed: 0'],axis=1)

obs_tab7['datetime'] =  pd.to_datetime(obs_tab7['datetime'], format='%Y-%m-%d %H:%M:%S')
obs_tab7['timepoint'] = obs_tab7['datetime'].dt.round('15min') #timepoint col needed in later manipulationn
print(obs_tab7.shape)
obs_tab7

##19,818,593 records

In [ ]:
#Import obs_tab8

obs_tab8= pd.read_csv("obs_tab8.csv")
obs_tab8= obs_tab8.drop(['Unnamed: 0'],axis=1)

obs_tab8['datetime'] =  pd.to_datetime(obs_tab8['datetime'], format='%Y-%m-%d %H:%M:%S')
obs_tab8['timepoint'] = obs_tab8['datetime'].dt.round('15min') #timepoint col needed in later manipulationn
print(obs_tab8.shape)
obs_tab8

##19,449,724 records

In [ ]:
#Import obs_tab9

obs_tab9= pd.read_csv("obs_tab9.csv")
obs_tab9= obs_tab9.drop(['Unnamed: 0'],axis=1)

obs_tab9['datetime'] =  pd.to_datetime(obs_tab9['datetime'], format='%Y-%m-%d %H:%M:%S')
obs_tab9['timepoint'] = obs_tab9['datetime'].dt.round('15min') #timepoint col needed in later manipulationn
print(obs_tab9.shape)
obs_tab9

##19,922,926 records

In [ ]:
#Import obs_tab10

obs_tab10= pd.read_csv("obs_tab10.csv")
obs_tab10= obs_tab10.drop(['Unnamed: 0'],axis=1)

obs_tab10['datetime'] =  pd.to_datetime(obs_tab10['datetime'], format='%Y-%m-%d %H:%M:%S')
obs_tab10['timepoint'] = obs_tab10['datetime'].dt.round('15min') #timepoint col needed in later manipulationn
print(obs_tab10.shape)
obs_tab10

##18,538,580 records

### 4) Find instances with 15min intervals containing readings for all 4 physiological parameters

In [ ]:
#Using obs_tab1
#List unique physiological variableids within 15min time intervals, per patientid 
##datetime needs to be in correct format for query to run

agg_var_1 = obs_tab1.groupby([pd.Grouper(key='datetime', freq='15Min'), 'patientid']).variableid.unique()
agg_var_1 = pd.DataFrame(data=agg_var_1)
agg_var_1 = agg_var_1.reset_index()
agg_var_1 = agg_var_1.sort_values(by=['patientid','datetime'], ascending=[True,True])

agg_var_1.to_csv('agg_var_1.csv')

print(agg_var_1.shape)
agg_var_1.head(5)

##715,533 records

In [ ]:
#Using obs_tab2
#List unique physiological variableids within 15min time intervals, per patientid 
##datetime needs to be in correct format for query to run

agg_var_2 = obs_tab2.groupby([pd.Grouper(key='datetime', freq='15Min'), 'patientid']).variableid.unique()
agg_var_2 = pd.DataFrame(data=agg_var_2)
agg_var_2 = agg_var_2.reset_index()
agg_var_2 = agg_var_2.sort_values(by=['patientid','datetime'], ascending=[True,True])

agg_var_2.to_csv('agg_var_2.csv')

print(agg_var_2.shape)
agg_var_2.head(5)

##720,062 records

In [ ]:
#Using obs_tab3
#List unique physiological variableids within 15min time intervals, per patientid 
#datetime needs to be in correct format for query to run

agg_var_3 = obs_tab3.groupby([pd.Grouper(key='datetime', freq='15Min'), 'patientid']).variableid.unique()
agg_var_3 = pd.DataFrame(data=agg_var_3)
agg_var_3 = agg_var_3.reset_index()
agg_var_3 = agg_var_3.sort_values(by=['patientid','datetime'], ascending=[True,True])

agg_var_3.to_csv('agg_var_3.csv')

print(agg_var_3.shape)
agg_var_3.head(5)

##737,329 records

In [ ]:
#Using obs_tab4
#List unique physiological variableids within 15min time intervals, per patientid 
#datetime needs to be in correct format for query to run

agg_var_4 = obs_tab4.groupby([pd.Grouper(key='datetime', freq='15Min'), 'patientid']).variableid.unique()
agg_var_4 = pd.DataFrame(data=agg_var_4)
agg_var_4 = agg_var_4.reset_index()
agg_var_4 = agg_var_4.sort_values(by=['patientid','datetime'], ascending=[True,True])

agg_var_4.to_csv('agg_var_4.csv')

print(agg_var_4.shape)
agg_var_4.head(5)

##722,235 records

In [ ]:
#Using obs_tab5
#List unique physiological variableids within 15min time intervals, per patientid 
#datetime needs to be in correct format for query to run

agg_var_5 = obs_tab5.groupby([pd.Grouper(key='datetime', freq='15Min'), 'patientid']).variableid.unique()
agg_var_5 = pd.DataFrame(data=agg_var_5)
agg_var_5 = agg_var_5.reset_index()
agg_var_5 = agg_var_5.sort_values(by=['patientid','datetime'], ascending=[True,True])

agg_var_5.to_csv('agg_var_5.csv')

print(agg_var_5.shape)
agg_var_5.head(5)

##712,740 records

In [ ]:
#Using obs_tab6
#List unique physiological variableids within 16min time intervals, per patientid 
#datetime needs to be in correct format for query to run

agg_var_6 = obs_tab6.groupby([pd.Grouper(key='datetime', freq='15Min'), 'patientid']).variableid.unique()
agg_var_6 = pd.DataFrame(data=agg_var_6)
agg_var_6 = agg_var_6.reset_index()
agg_var_6 = agg_var_6.sort_values(by=['patientid','datetime'], ascending=[True,True])

agg_var_6.to_csv('agg_var_6.csv')

print(agg_var_6.shape)
agg_var_6.head(6)

##729,026 records

In [ ]:
#Using obs_tab7
#List unique physiological variableids within 17min time intervals, per patientid 
#datetime needs to be in correct format for query to run

agg_var_7 = obs_tab7.groupby([pd.Grouper(key='datetime', freq='15Min'), 'patientid']).variableid.unique()
agg_var_7 = pd.DataFrame(data=agg_var_7)
agg_var_7 = agg_var_7.reset_index()
agg_var_7 = agg_var_7.sort_values(by=['patientid','datetime'], ascending=[True,True])

agg_var_7.to_csv('agg_var_7.csv')

print(agg_var_7.shape)
agg_var_7.head(7)

##734,652 records

In [ ]:
#Using obs_tab8
#List unique physiological variableids within 18min time intervals, per patientid 
#datetime needs to be in correct format for query to run

agg_var_8 = obs_tab8.groupby([pd.Grouper(key='datetime', freq='15Min'), 'patientid']).variableid.unique()
agg_var_8 = pd.DataFrame(data=agg_var_8)
agg_var_8 = agg_var_8.reset_index()
agg_var_8 = agg_var_8.sort_values(by=['patientid','datetime'], ascending=[True,True])

agg_var_8.to_csv('agg_var_8.csv')

print(agg_var_8.shape)
agg_var_8.head(8)

##712,585 records

In [ ]:
#Using obs_tab9
#List unique physiological variableids within 19min time intervals, per patientid 
#datetime needs to be in correct format for query to run

agg_var_9 = obs_tab9.groupby([pd.Grouper(key='datetime', freq='15Min'), 'patientid']).variableid.unique()
agg_var_9 = pd.DataFrame(data=agg_var_9)
agg_var_9 = agg_var_9.reset_index()
agg_var_9 = agg_var_9.sort_values(by=['patientid','datetime'], ascending=[True,True])

agg_var_9.to_csv('agg_var_9.csv')

print(agg_var_9.shape)
agg_var_9.head(9)

##734,800 records

In [ ]:
#Using obs_tab10
#List unique physiological variableids within 110min time intervals, per patientid 
#datetime needs to be in correct format for query to run

agg_var_10 = obs_tab10.groupby([pd.Grouper(key='datetime', freq='15Min'), 'patientid']).variableid.unique()
agg_var_10 = pd.DataFrame(data=agg_var_10)
agg_var_10 = agg_var_10.reset_index()
agg_var_10 = agg_var_10.sort_values(by=['patientid','datetime'], ascending=[True,True])

agg_var_10.to_csv('agg_var_10.csv')

print(agg_var_10.shape)
agg_var_10.head(10)

##685,927 records

In [ ]:
#Import
agg_var_1 = pd.read_csv("agg_var_1.csv")
agg_var_1 = agg_var_1.drop(['Unnamed: 0'],axis=1)

agg_var_2 = pd.read_csv("agg_var_2.csv")
agg_var_2 = agg_var_2.drop(['Unnamed: 0'],axis=1)

agg_var_3 = pd.read_csv("agg_var_3.csv")
agg_var_3 = agg_var_3.drop(['Unnamed: 0'],axis=1)

agg_var_4 = pd.read_csv("agg_var_4.csv")
agg_var_4 = agg_var_4.drop(['Unnamed: 0'],axis=1)

agg_var_5 = pd.read_csv("agg_var_5.csv")
agg_var_5 = agg_var_5.drop(['Unnamed: 0'],axis=1)


In [ ]:
#Import
agg_var_1 = pd.read_csv("agg_var_1.csv")
agg_var_1 = agg_var_1.drop(['Unnamed: 0'],axis=1)

agg_var_2 = pd.read_csv("agg_var_2.csv")
agg_var_2 = agg_var_2.drop(['Unnamed: 0'],axis=1)

agg_var_3 = pd.read_csv("agg_var_3.csv")
agg_var_3 = agg_var_3.drop(['Unnamed: 0'],axis=1)

agg_var_4 = pd.read_csv("agg_var_4.csv")
agg_var_4 = agg_var_4.drop(['Unnamed: 0'],axis=1)

agg_var_5 = pd.read_csv("agg_var_5.csv")
agg_var_5 = agg_var_5.drop(['Unnamed: 0'],axis=1)

agg_var_6 = pd.read_csv("agg_var_6.csv")
agg_var_6 = agg_var_6.drop(['Unnamed: 0'],axis=1)

agg_var_7 = pd.read_csv("agg_var_7.csv")
agg_var_7 = agg_var_7.drop(['Unnamed: 0'],axis=1)

agg_var_8 = pd.read_csv("agg_var_8.csv")
agg_var_8 = agg_var_8.drop(['Unnamed: 0'],axis=1)

agg_var_9 = pd.read_csv("agg_var_9.csv")
agg_var_9 = agg_var_9.drop(['Unnamed: 0'],axis=1)

agg_var_10 = pd.read_csv("agg_var_10.csv")
agg_var_10 = agg_var_10.drop(['Unnamed: 0'],axis=1)

In [ ]:
#Concatenate 
#List unique physiological variableids within ALL 15min time intervals, per patientid 

frames = [agg_var_1, agg_var_2, agg_var_3, agg_var_4, agg_var_5, 
          agg_var_6, agg_var_7, agg_var_8, agg_var_9, agg_var_10]

agg_var_full = pd.concat(frames)

agg_var_full.to_csv('agg_var_full.csv')

print(agg_var_full.shape)
agg_var_full

##7,204,889 records

In [ ]:
#if necessary

agg_var_full = pd.read_csv("agg_var_full.csv")
agg_var_full = agg_var_full.drop(['Unnamed: 0'],axis=1)

print(agg_var_full.shape)

agg_var_full['datetime'] =  pd.to_datetime(agg_var_full['datetime'], format='%Y-%m-%d %H:%M:%S.%f')

agg_var_full.head()

In [ ]:
#Filter agg_var_full for all instances that contain a reading for FiO2

#FiO2
df_mod = agg_var_full[agg_var_full.variableid.astype(str).str.contains('2010')]
print(df_mod.shape)
df_mod

##3,040,811 records

In [ ]:
#Filter df_mod for all instances that contain a reading for SpO2

df_mod = df_mod[df_mod.variableid.astype(str).str.contains('4000|8280')]
print(df_mod.shape)
df_mod

##3,021,057 records

In [ ]:
#Filter df_mod for all instances that contain a reading for HR

df_mod = df_mod[df_mod.variableid.astype(str).str.contains('200')]
print(df_mod.shape)
df_mod

##3,020,405 records

In [ ]:
#Filter df_mod for all instances that contain a reading for MAP

df_mod = df_mod[df_mod.variableid.astype(str).str.contains('110|610')]

df_mod.to_csv('df_mod.csv')

print(df_mod.shape)
df_mod

##3,004,013 records
##There are 3,004,013 unique instances which contain readings for all 4 physiological parameters
##This is our pool of relevant (patientid + timepoint) combinations

In [ ]:
#if necessary - import
df_mod = pd.read_csv("df_mod.csv")
df_mod = df_mod.drop(['Unnamed: 0'],axis=1)

print(df_mod.shape)

df_mod['datetime'] =  pd.to_datetime(df_mod['datetime'], format='%Y-%m-%d %H:%M:%S')

df_mod.head()

In [ ]:
#Check unique ICU admissions

df_mod['patientid'].nunique()

##20,289 unique ICU admissions within above pool of relevant instances

In [ ]:
df_mod.groupby(['datetime', 'patientid']).ngroups

#confirms there are 3,004,013 unique combinations of (15min time interval + patientid) that have readings for 4 phys params

### 5) Find instances with readings for Adrenaline/Noradrenaline AND all 4 physiological parameters

In [ ]:
#Merge df_mod and agg_drugs
##Pool of relevant instances (unique timepoint + patientid)

agg_pool = pd.merge(df_mod, agg_drugs,  how='left', left_on=['datetime','patientid'], right_on = ['givenat','patientid'])
agg_pool = agg_pool.drop(['givenat'],axis=1)
agg_pool.rename({'datetime': 'timepoint'}, axis=1, inplace=True)

agg_pool['timepoint'] =  pd.to_datetime(agg_pool['timepoint'], format='%Y-%m-%d %H:%M:%S')

agg_pool.to_csv('agg_pool.csv')

print(agg_pool.shape)
agg_pool

#3,004,013 instances that have 4+ readings for phyiological parameters 

In [ ]:
#if necessary - convert timepoint to datetime format
agg_pool['timepoint'] =  pd.to_datetime(agg_pool['timepoint'], format='%Y-%m-%d %H:%M:%S')
agg_pool.dtypes

In [ ]:
#Check how many instances (unique patientid + timepoint) are common between df_mod and agg_drugs 

agg_merge = pd.merge(agg_drugs, df_mod,  how='left', left_on=['givenat','patientid'], right_on = ['datetime','patientid'])
agg_merge = agg_merge.drop(['datetime'],axis=1)
agg_merge.rename({'givenat': 'timepoint'}, axis=1, inplace=True)

agg_merge = agg_merge[agg_merge['variableid'].notna()]

print(agg_merge.shape)
agg_merge.head()

#610,917 instances with non-zero readings for Adrenaline/Noradrenaline AND all 4 physiological parameters

### 6) For (patientid + timepoint) present in agg_pool, find all instances of physiological parameter readings

In [ ]:
#Filter obs_tab1 for only relevant (patientid + timepoint) combinations (ie. those present in agg_pool dataframe)

obs1_mod = pd.merge(obs_tab1, agg_pool,  how='left', left_on=['timepoint','patientid'], right_on = ['timepoint','patientid'])
obs1_mod = obs1_mod[obs1_mod['variableid_y'].notna()]
obs1_mod.rename({'variableid_x': 'variable'}, axis=1, inplace=True)

obs1 = obs1_mod[['patientid','timepoint', 'variable', 'value']]
obs1 = obs1.sort_values(by=['patientid','timepoint','variable'], ascending=[True,True,True])

#Replace variable numbers by names
obs1['variable'] = obs1['variable'].replace({ 2010 : "FiO2", 4000 : "SpO2", 8280: "SpO2", 200 : "HR", 
                                                    110 : "MAP", 610 : "MAP"})

#List all variables per unique (patientid + timepoint)
##Primary Key = (patientid + timepoint + variable)
obs1 = obs1.groupby(['patientid','timepoint', 'variable'])['value'].apply(list)

obs1 = pd.DataFrame(data=obs1)
obs1 = obs1.reset_index()
obs1.to_csv('obs1.csv')
print(obs1.shape)
obs1

##1,161,327 records

In [ ]:
#Filter obs_tab2 for only relevant (patientid + timepoint) combinations (ie. those present in agg_pool dataframe)

obs2_mod = pd.merge(obs_tab2, agg_pool,  how='left', left_on=['timepoint','patientid'], right_on = ['timepoint','patientid'])
obs2_mod = obs2_mod[obs2_mod['variableid_y'].notna()]
obs2_mod.rename({'variableid_x': 'variable'}, axis=1, inplace=True)

obs2 = obs2_mod[['patientid','timepoint', 'variable', 'value']]
obs2 = obs2.sort_values(by=['patientid','timepoint','variable'], ascending=[True,True,True])

#Replace variable numbers by names
obs2['variable'] = obs2['variable'].replace({ 2010 : "FiO2", 4000 : "SpO2", 8280: "SpO2", 200 : "HR", 
                                                    110 : "MAP", 610 : "MAP"})

#List all variables per unique (patientid + timepoint)
obs2 = obs2.groupby(['patientid','timepoint', 'variable'])['value'].apply(list)

obs2 = pd.DataFrame(data=obs2)
obs2 = obs2.reset_index()
obs2.to_csv('obs2.csv')
print(obs2.shape)
obs2

#1,218,664 records

In [ ]:
#Filter obs_tab3 for only relevant (patientid + timepoint) combinations (ie. those present in agg_pool dataframe)

obs3_mod = pd.merge(obs_tab3, agg_pool,  how='left', left_on=['timepoint','patientid'], right_on = ['timepoint','patientid'])
obs3_mod = obs3_mod[obs3_mod['variableid_y'].notna()]
obs3_mod.rename({'variableid_x': 'variable'}, axis=1, inplace=True)

obs3 = obs3_mod[['patientid','timepoint', 'variable', 'value']]
obs3 = obs3.sort_values(by=['patientid','timepoint','variable'], ascending=[True,True,True])

#Replace variable numbers by names
obs3['variable'] = obs3['variable'].replace({ 2010 : "FiO2", 4000 : "SpO2", 8280: "SpO2", 200 : "HR", 
                                                    110 : "MAP", 610 : "MAP"})

#List all variables per unique (patientid + timepoint)
obs3 = obs3.groupby(['patientid','timepoint', 'variable'])['value'].apply(list)

obs3 = pd.DataFrame(data=obs3)
obs3 = obs3.reset_index()
obs3.to_csv('obs3.csv')
print(obs3.shape)
obs3

#1,249,820 records

In [ ]:
#Filter obs_tab4 for only relevant (patientid + timepoint) combinations (ie. those present in agg_pool dataframe)

obs4_mod = pd.merge(obs_tab4, agg_pool,  how='left', left_on=['timepoint','patientid'], right_on = ['timepoint','patientid'])
obs4_mod = obs4_mod[obs4_mod['variableid_y'].notna()]
obs4_mod.rename({'variableid_x': 'variable'}, axis=1, inplace=True)

obs4 = obs4_mod[['patientid','timepoint', 'variable', 'value']]
obs4 = obs4.sort_values(by=['patientid','timepoint','variable'], ascending=[True,True,True])

#Replace variable numbers by names
obs4['variable'] = obs4['variable'].replace({ 2010 : "FiO2", 4000 : "SpO2", 8280: "SpO2", 200 : "HR", 
                                                    110 : "MAP", 610 : "MAP"})

#List all variables per unique (patientid + timepoint)
obs4 = obs4.groupby(['patientid','timepoint', 'variable'])['value'].apply(list)

obs4 = pd.DataFrame(data=obs4)
obs4 = obs4.reset_index()
obs4.to_csv('obs4.csv')
print(obs4.shape)
obs4

#1,169,874 records

In [ ]:
#Filter obs_tab5 for only relevant (patientid + timepoint) combinations (ie. those present in agg_pool dataframe)

obs5_mod = pd.merge(obs_tab5, agg_pool,  how='left', left_on=['timepoint','patientid'], right_on = ['timepoint','patientid'])
obs5_mod = obs5_mod[obs5_mod['variableid_y'].notna()]
obs5_mod.rename({'variableid_x': 'variable'}, axis=1, inplace=True)

obs5 = obs5_mod[['patientid','timepoint', 'variable', 'value']]
obs5 = obs5.sort_values(by=['patientid','timepoint','variable'], ascending=[True,True,True])

#Replace variable numbers by names
obs5['variable'] = obs5['variable'].replace({ 2010 : "FiO2", 4000 : "SpO2", 8280: "SpO2", 200 : "HR", 
                                                    110 : "MAP", 610 : "MAP"})

#List all variables per unique (patientid + timepoint)
obs5 = obs5.groupby(['patientid','timepoint', 'variable'])['value'].apply(list)

obs5 = pd.DataFrame(data=obs5)
obs5 = obs5.reset_index()
obs5.to_csv('obs5.csv')
print(obs5.shape)
obs5

##1,175,952 records

In [ ]:
#Filter obs_tab6 for only relevant (patientid + timepoint) combinations (ie. those present in agg_pool dataframe)

obs6_mod = pd.merge(obs_tab6, agg_pool,  how='left', left_on=['timepoint','patientid'], right_on = ['timepoint','patientid'])
obs6_mod = obs6_mod[obs6_mod['variableid_y'].notna()]
obs6_mod.rename({'variableid_x': 'variable'}, axis=1, inplace=True)

obs6 = obs6_mod[['patientid','timepoint', 'variable', 'value']]
obs6 = obs6.sort_values(by=['patientid','timepoint','variable'], ascending=[True,True,True])

#Replace variable numbers by names
obs6['variable'] = obs6['variable'].replace({ 2010 : "FiO2", 4000 : "SpO2", 8280: "SpO2", 200 : "HR", 
                                                    110 : "MAP", 610 : "MAP"})

#List all variables per unique (patientid + timepoint)
obs6 = obs6.groupby(['patientid','timepoint', 'variable'])['value'].apply(list)

obs6 = pd.DataFrame(data=obs6)
obs6 = obs6.reset_index()
obs6.to_csv('obs6.csv')
print(obs6.shape)
obs6

##1,181,113 records

In [ ]:
#Filter obs_tab7 for only relevant (patientid + timepoint) combinations (ie. those present in agg_pool dataframe)

obs7_mod = pd.merge(obs_tab7, agg_pool,  how='left', left_on=['timepoint','patientid'], right_on = ['timepoint','patientid'])
obs7_mod = obs7_mod[obs7_mod['variableid_y'].notna()]
obs7_mod.rename({'variableid_x': 'variable'}, axis=1, inplace=True)

obs7 = obs7_mod[['patientid','timepoint', 'variable', 'value']]
obs7 = obs7.sort_values(by=['patientid','timepoint','variable'], ascending=[True,True,True])

#Replace variable numbers by names

obs7['variable'] = obs7['variable'].replace({ 2010 : "FiO2", 4000 : "SpO2", 8280: "SpO2", 200 : "HR", 
                                                    110 : "MAP", 610 : "MAP"})

#List all variables per unique (patientid + timepoint)
##Primary Key = (patientid + timepoint + variable)
obs7 = obs7.groupby(['patientid','timepoint', 'variable'])['value'].apply(list)

obs7 = pd.DataFrame(data=obs7)
obs7 = obs7.reset_index()
obs7.to_csv('obs7.csv')
print(obs7.shape)
obs7

##1,250,260 records

In [ ]:
#Filter obs_tab8 for only relevant (patientid + timepoint) combinations (ie. those present in agg_pool dataframe)

obs8_mod = pd.merge(obs_tab8, agg_pool,  how='left', left_on=['timepoint','patientid'], right_on = ['timepoint','patientid'])
obs8_mod = obs8_mod[obs8_mod['variableid_y'].notna()]
obs8_mod.rename({'variableid_x': 'variable'}, axis=1, inplace=True)

obs8 = obs8_mod[['patientid','timepoint', 'variable', 'value']]
obs8 = obs8.sort_values(by=['patientid','timepoint','variable'], ascending=[True,True,True])

#Replace variable numbers by names

obs8['variable'] = obs8['variable'].replace({ 2010 : "FiO2", 4000 : "SpO2", 8280: "SpO2", 200 : "HR", 
                                                    110 : "MAP", 610 : "MAP"})

#List all variables per unique (patientid + timepoint)
##Primary Key = (patientid + timepoint + variable)
obs8 = obs8.groupby(['patientid','timepoint', 'variable'])['value'].apply(list)

obs8 = pd.DataFrame(data=obs8)
obs8 = obs8.reset_index()
obs8.to_csv('obs8.csv')
print(obs8.shape)
obs8

##1,215,066 records

In [ ]:
#Filter obs_tab9 for only relevant (patientid + timepoint) combinations (ie. those present in agg_pool dataframe)

obs9_mod = pd.merge(obs_tab9, agg_pool,  how='left', left_on=['timepoint','patientid'], right_on = ['timepoint','patientid'])
obs9_mod = obs9_mod[obs9_mod['variableid_y'].notna()]
obs9_mod.rename({'variableid_x': 'variable'}, axis=1, inplace=True)

obs9 = obs9_mod[['patientid','timepoint', 'variable', 'value']]
obs9 = obs9.sort_values(by=['patientid','timepoint','variable'], ascending=[True,True,True])

#Replace variable numbers by names

obs9['variable'] = obs9['variable'].replace({ 2010 : "FiO2", 4000 : "SpO2", 8280: "SpO2", 200 : "HR", 
                                                    110 : "MAP", 610 : "MAP"})

#List all variables per unique (patientid + timepoint)
##Primary Key = (patientid + timepoint + variable)
obs9 = obs9.groupby(['patientid','timepoint', 'variable'])['value'].apply(list)

obs9 = pd.DataFrame(data=obs9)
obs9 = obs9.reset_index()
obs9.to_csv('obs9.csv')
print(obs9.shape)
obs9

##1,233,930 records

In [ ]:
#Filter obs_tab10 for only relevant (patientid + timepoint) combinations (ie. those present in agg_pool dataframe)

obs10_mod = pd.merge(obs_tab10, agg_pool,  how='left', left_on=['timepoint','patientid'], right_on = ['timepoint','patientid'])
obs10_mod = obs10_mod[obs10_mod['variableid_y'].notna()]
obs10_mod.rename({'variableid_x': 'variable'}, axis=1, inplace=True)

obs10 = obs10_mod[['patientid','timepoint', 'variable', 'value']]
obs10 = obs10.sort_values(by=['patientid','timepoint','variable'], ascending=[True,True,True])

#Replace variable numbers by names

obs10['variable'] = obs10['variable'].replace({ 2010 : "FiO2", 4000 : "SpO2", 8280: "SpO2", 200 : "HR", 
                                                    110 : "MAP", 610 : "MAP"})

#List all variables per unique (patientid + timepoint)
##Primary Key = (patientid + timepoint + variable)
obs10 = obs10.groupby(['patientid','timepoint', 'variable'])['value'].apply(list)

obs10 = pd.DataFrame(data=obs10)
obs10 = obs10.reset_index()
obs10.to_csv('obs10.csv')
print(obs10.shape)
obs10

##1,099,819 records

In [ ]:
#Import all obsx
obs1 = pd.read_csv("obs1.csv")
obs1 = obs1.drop(['Unnamed: 0'],axis=1)

obs2 = pd.read_csv("obs2.csv")
obs2 = obs2.drop(['Unnamed: 0'],axis=1)

obs3 = pd.read_csv("obs3.csv")
obs3 = obs3.drop(['Unnamed: 0'],axis=1)

obs4 = pd.read_csv("obs4.csv")
obs4 = obs4.drop(['Unnamed: 0'],axis=1)

obs5 = pd.read_csv("obs5.csv")
obs5 = obs5.drop(['Unnamed: 0'],axis=1)

obs6 = pd.read_csv("obs6.csv")
obs6 = obs6.drop(['Unnamed: 0'],axis=1)

obs7 = pd.read_csv("obs7.csv")
obs7 = obs7.drop(['Unnamed: 0'],axis=1)

obs8 = pd.read_csv("obs8.csv")
obs8 = obs8.drop(['Unnamed: 0'],axis=1)

obs9 = pd.read_csv("obs9.csv")
obs9 = obs9.drop(['Unnamed: 0'],axis=1)

obs10 = pd.read_csv("obs10.csv")
obs10 = obs10.drop(['Unnamed: 0'],axis=1)

obs10.head()

In [ ]:
#Merge 

frames = [obs1, obs2, obs3, obs4, obs5, obs6, obs7, obs8, obs9, obs10]

params_comp= pd.concat(frames)

params_comp.to_csv('params_comp.csv')

print(params_comp.shape)
params_comp

##11,955,825 records 

In [ ]:
#check unique ICU admissions in params_comp

params_comp['patientid'].nunique()

##20,286 unique ICU admissions

In [ ]:
#Import

params_comp = pd.read_csv("params_comp.csv")
params_comp = params_comp.drop(['Unnamed: 0'],axis=1)

print(params_comp.shape)
params_comp.head()

In [ ]:
params_comp.dtypes

### 7) Check all variable values are in acceptable range with compatible units

In [ ]:
#Check format of 'value' column

for i, l in enumerate(params_comp["value"]):
    print("list",i,"is",type(l))
    
##all rows in 'value' column are in str format

In [ ]:
#if necessary - convert timepoint to datetime format
params_comp['timepoint'] =  pd.to_datetime(params_comp['timepoint'], format='%Y-%m-%d %H:%M:%S')
params_comp.dtypes 

In [ ]:
#Check all FiO2 values are in acceptable range

check_fio2 = params_comp.copy(deep=True)

check_fio2 = check_fio2[check_fio2['variable']=='FiO2']
check_fio2['fio2_check'] = check_fio2.value.apply(lambda x: [ all(c >= 21 and c <=100 for c in x)])
check_fio2['acceptable_range'] = check_fio2.fio2_check.apply(lambda x: 'Yes' if True in x else 'No')

check_fio2

##2,983,684 records

In [ ]:
len(check_fio2[check_fio2['acceptable_range']=='No'])

#2,439 instances containing FiO2 values outside acceptable range 

In [ ]:
#Create new column FiO2 values that are in acceptable range

check_fio2['inrange_val_list'] = check_fio2.value.apply(lambda x: [ c for c in x if 21 <= c <= 100])
check_fio2 = check_fio2.drop(['value','fio2_check','acceptable_range'],axis=1)

check_fio2 = check_fio2.dropna()

print(check_fio2.shape)
check_fio2.head()

##2,983,684 records

In [ ]:
#Confirm all FiO2 values are in acceptable range

check_fio2['fio2_check'] = check_fio2.inrange_val_list.apply(lambda x: [ all(c >= 21 and c <=100 for c in x)])
check_fio2['acceptable_range'] = check_fio2.fio2_check.apply(lambda x: 'Yes' if True in x else 'No')

check_fio2['acceptable_range'].unique()

##All FiO2 values are now in acceptable range 

In [ ]:
#Drop unncessary columns

check_fio2 = check_fio2.drop(['fio2_check','acceptable_range'],axis=1)

check_fio2.to_csv('check_fio2.csv')

print(check_fio2.shape)
check_fio2.head()

##2,983,684 records

In [ ]:
#Check all SpO2 values are in acceptable range

check_spo2 = params_comp.copy(deep=True)

check_spo2 = check_spo2[check_spo2['variable']=='SpO2']
check_spo2['spo2_check'] = check_spo2.value.apply(lambda x: [ all(c >= 0 and c <=100 for c in x)])
check_spo2['acceptable_range'] = check_spo2.spo2_check.apply(lambda x: 'Yes' if True in x else 'No')

check_spo2

##2,988,937 records

In [ ]:
len(check_spo2[check_spo2['acceptable_range']=='No'])

#14 instances contain SpO2 values outside acceptable ranges 

In [ ]:
#Create new column SpO2 values that are in acceptable range

check_spo2['inrange_val_list'] = check_spo2.value.apply(lambda x: [ c for c in x if 0 <= c <= 100])
check_spo2 = check_spo2.drop(['value','spo2_check','acceptable_range'],axis=1)

check_spo2 = check_spo2.dropna()

print(check_spo2.shape)
check_spo2.head()

##2,988,937 records

In [ ]:
#Confirm all SpO2 values are in acceptable range

check_spo2['spo2_check'] = check_spo2.inrange_val_list.apply(lambda x: [ all(c >= 0 and c <=100 for c in x)])
check_spo2['acceptable_range'] = check_spo2.spo2_check.apply(lambda x: 'Yes' if True in x else 'No')

check_spo2['acceptable_range'].unique()

##All SpO2 values are now in acceptable range 

In [ ]:
#Drop unncessary columns

check_spo2 = check_spo2.drop(['spo2_check','acceptable_range'],axis=1)

check_spo2.to_csv('check_spo2.csv')

print(check_spo2.shape)
check_spo2.head()

##2,988,937 records

In [ ]:
#Check all MAP values are in acceptable range

check_map = params_comp.copy(deep=True)

check_map = check_map[check_map['variable']=='MAP']
check_map['map_check'] = check_map.value.apply(lambda x: [ all(c >= 0 and c <=200 for c in x)])
check_map['acceptable_range'] = check_map.map_check.apply(lambda x: 'Yes' if True in x else 'No')

check_map

##2,989,610 records

In [ ]:
len(check_map[check_map['acceptable_range']=='No'])

#61,820 instances contain an MAP reading outside the acceptable range 

In [ ]:
#Create new column MAP values that are in acceptable range

check_map['inrange_val_list'] = check_map.value.apply(lambda x: [ c for c in x if 0 <= c <= 200])
check_map = check_map.drop(['value','map_check','acceptable_range'],axis=1)

check_map = check_map.dropna()

print(check_map.shape)
check_map.head()

##2,989,610 records

In [ ]:
#Confirm all MAP values are in acceptable range

check_map['map_check'] = check_map.inrange_val_list.apply(lambda x: [ all(c >= 0 and c <=200 for c in x)])
check_map['acceptable_range'] = check_map.map_check.apply(lambda x: 'Yes' if True in x else 'No')

check_map['acceptable_range'].unique()

##All MAP values are now in acceptable range

In [ ]:
#Drop unncessary columns

check_map = check_map.drop(['map_check','acceptable_range'],axis=1)

check_map.to_csv('check_map.csv')

print(check_map.shape)
check_map.head()

##2,989,610 records

In [ ]:
#Check all HR values are in acceptable range

check_hr = params_comp.copy(deep=True)

check_hr = check_hr[check_hr['variable']=='HR']
check_hr['hr_check'] = check_hr.value.apply(lambda x: [ all(c >= 0 and c <=300 for c in x)])
check_hr['acceptable_range'] = check_hr.hr_check.apply(lambda x: 'Yes' if True in x else 'No')
check_hr

##2,993,594 records

In [ ]:
len(check_hr[check_hr['acceptable_range']=='No'])

#All HR values are in acceptable range 

In [ ]:
#Drop unncessary columns

check_hr = check_hr.drop(['hr_check','acceptable_range'],axis=1)
check_hr.rename({'value': 'inrange_val_list'}, axis=1, inplace=True)

check_hr.to_csv('check_hr.csv')

print(check_hr.shape)
check_hr.head()

In [ ]:
print(check_fio2.columns)
print(check_spo2.columns)
print(check_map.columns)
print(check_hr.columns)

In [ ]:
#Concatenate check,fio2, check_spo2, check_map check_hr

frames = [check_fio2, check_spo2, check_map, check_hr]

params_comp = pd.concat(frames)

params_comp.sort_values(by=['patientid','timepoint'], ascending=[True,True])

params_comp.to_csv('params_comp2.csv')

print(params_comp.shape)
params_comp

##11,955,825 records

### 8)  Extract minimum, maximum and average variable values per unique instance

In [ ]:
#Extract minimum and maximum variable values

params_comp['min_val'] = params_comp.inrange_val_list.apply(lambda x: min(x, default=np.nan))
params_comp['max_val'] = params_comp.inrange_val_list.apply(lambda x: max(x, default=np.nan))

params_comp.to_csv('params_comp3.csv')

print(params_comp.shape)
params_comp.head()

##11,955,825 records

In [ ]:
params_comp.dtypes

In [ ]:
#Extract average variable value

import statistics

params_comp = params_comp.dropna() #drop all rows containing null values (indicates empty value list)

params_comp['avg_val'] = params_comp.inrange_val_list.apply(lambda x: round(statistics.mean(x),1))

params_comp.to_csv('params_comp4.csv')

print(params_comp.shape)
params_comp.head()

##11,955,670 records

### 9) Remove instances where % variation is larger than defined cut-off

As advised by medical advisor, a large variation should not be observed within a such a short time period, and is indicative of a data error. <br>
•	**% variation** = (( max_val – min_val ) / min_val ) x 100


In [ ]:
#Check % variation of variable readings for each instance

params_comp['variation(%)'] = ((params_comp['max_val']-params_comp['min_val'])/params_comp['min_val'])*100

print(params_comp.shape)
params_comp.head() 

##11,955,670 records

In [ ]:
#Remove instances where the % variation of values within each 15-minute timepoint is within is greater than defined cut-off
###Instances to remove - cut-offs advised by X (medical advisor to project)

params_comp = params_comp[((params_comp['variable']=='FiO2') & (params_comp['variation(%)']<=5))|
                          ((params_comp['variable']=='SpO2') & (params_comp['variation(%)']<=10))|
                          ((params_comp['variable']=='MAP') & (params_comp['variation(%)']<=25))|
                          ((params_comp['variable']=='HR') & (params_comp['variation(%)']<=25))]


params_comp.to_csv('params_comp5.csv')

print(params_comp.shape)
params_comp.head() 

#11,170,443 records

### 10) Check average parameter values match training data units and ranges

In [ ]:
#Extract relevant columns from params_comp
##Reformat dataframe to similar structure as Glasgow training data, using average parameter values 

avg_params = params_comp[['patientid','timepoint','variable','avg_val']]

avg_params = avg_params.groupby(['patientid', 'timepoint', 'variable'])['avg_val'].sum().unstack('variable')

avg_params = avg_params.reset_index()
avg_params = avg_params.rename_axis(None, axis=1)
avg_params = avg_params.dropna() #drop any rows that contain null values 

print(avg_params.shape)
avg_params

##2,022,310 records
##need to convert FiO2 units from % to decFrc (HR, MAP, SpO2 have correct units)

In [ ]:
#Convert FiO2 to decFrc (compatible with Glasgow training data)

avg_params['FiO2']=round((avg_params['FiO2']/100),4)
print(avg_params.shape)
avg_params

#2,022,310 records

In [ ]:
##Confirm all average values are within acceptable range

avg_params = avg_params[avg_params['FiO2']>=0.21]
avg_params = avg_params[avg_params['FiO2']<=1.0]

avg_params = avg_params[avg_params['SpO2']>0]
avg_params = avg_params[avg_params['SpO2']<=100]

avg_params = avg_params[avg_params['HR']>0]
avg_params = avg_params[avg_params['HR']<=300]

avg_params = avg_params[avg_params['MAP']>0]
avg_params = avg_params[avg_params['MAP']<=200]

avg_params.to_csv('avg_params.csv')
print(avg_params.shape)
avg_params

##2,022,310 records

In [ ]:
#Check number of unique ICU admissions in avg_params

avg_params['patientid'].nunique()

##20,073 unique ICU admissions

### 11) Create HiRID external dataset using Average parameter values 

In [ ]:
print(avg_params.shape)
avg_params.head()

##2,022,310 records

In [ ]:
drugs_rate.head()

In [ ]:
#if necessary - convert timepoint to correct datetime format
drugs_rate['timepoint'] =  pd.to_datetime(drugs_rate['timepoint'], format='%Y-%m-%d %H:%M:%S')
drugs_rate.dtypes

In [ ]:
avg_params['timepoint'] =  pd.to_datetime(avg_params['timepoint'], format='%Y-%m-%d %H:%M:%S')
avg_params.dtypes

In [ ]:
#Merge avg_params and drugs_rate on (patientid + timepoint)

comp_avg_params = pd.merge(avg_params, drugs_rate, how='left', on=['patientid','timepoint'])
comp_avg_params = comp_avg_params[['patientid','timepoint','Adrenaline','Noradrenaline','FiO2','SpO2','MAP','HR']]

comp_avg_params['rowID']=comp_avg_params.index #create new index column

#move index col to start
col_at_start = ['rowID']
comp_avg_params = comp_avg_params[[c for c in col_at_start if c in comp_avg_params] + [c for c in comp_avg_params if c not in col_at_start]]

#replace null with 0 in drug fields (as blank value indicates value=0)
comp_avg_params['Adrenaline'] = comp_avg_params['Adrenaline'].replace(np.nan, 0)
comp_avg_params['Noradrenaline'] = comp_avg_params['Noradrenaline'].replace(np.nan, 0)

comp_avg_params.to_csv('comp_avg_params.csv')

print(comp_avg_params.shape)
comp_avg_params

##2,022,310 records

In [ ]:
#Check number of non-zero drug fields

print(len(comp_avg_params[comp_avg_params['Adrenaline']>0]))
print(len(comp_avg_params[comp_avg_params['Noradrenaline']>0]))

### 12) Find discharge & death times for each unique ICU admission

As the HiRID databse does not contain data for an ICU discharge/death time, the last Heart Rate reading was used as the discharge time (if discharge_status=alive) or death time (if discharge_status=dead). This approach was recommended by the HiRID v1.1.1 database corresponding author, Martin Faltys.

In [ ]:
#Connect to observations_table_1 - find all HR readings

hr_tab1 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_1 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab1['row_idx']=hr_tab1.index #create new index column

print(hr_tab1.shape)
hr_tab1.head()

##6,076,840 records

In [ ]:
hr_tab1_patlist = list(hr_tab1['patientid'].unique())
len(hr_tab1_patlist)

##3386 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab1 -> this is equivalent to discharge/death time 

temp = hr_tab1.groupby('patientid')
dtime_tab1 = temp.agg(max_date=('datetime', np.max))
dtime_tab1 = dtime_tab1.reset_index()

dtime_tab1.to_csv('dtime_tab1.csv')
dtime_tab1

#max_date = discharge time/ death time for all patientids in hr_tab1

In [ ]:
#Connect to observations_table_2 - find all HR readings

hr_tab2 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_2 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab2['row_idx']=hr_tab2.index #create new index column

print(hr_tab2.shape)
hr_tab2.head()

##5,943,365 records

In [ ]:
hr_tab2_patlist = list(hr_tab2['patientid'].unique())
len(hr_tab2_patlist)

##3391 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab2

temp = hr_tab2.groupby('patientid')
dtime_tab2 = temp.agg(max_date=('datetime', np.max))
dtime_tab2 = dtime_tab2.reset_index()

dtime_tab2.to_csv('dtime_tab2.csv')

dtime_tab2

#max_date = discharge time/ death time for all patientids in hr_tab2

In [ ]:
#Connect to observations_table_3 - find all HR readings

hr_tab3 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_3 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab3['row_idx']=hr_tab3.index #create new index column

print(hr_tab3.shape)
hr_tab3.head()

##6,157,038 records

In [ ]:
hr_tab3_patlist = list(hr_tab3['patientid'].unique())
len(hr_tab3_patlist)

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab3

temp = hr_tab3.groupby('patientid')
dtime_tab3 = temp.agg(max_date=('datetime', np.max))
dtime_tab3 = dtime_tab3.reset_index()

dtime_tab3.to_csv('dtime_tab3.csv')
dtime_tab3

#max_date = discharge time/ death time for all patientids in hr_tab3

In [ ]:
#Connect to observations_table_4 - find all HR readings

hr_tab4 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_4 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab4['row_idx']=hr_tab4.index #create new index column

print(hr_tab4.shape)
hr_tab4.head()

##6,072,003 records

In [ ]:
hr_tab4_patlist = list(hr_tab4['patientid'].unique())
len(hr_tab4_patlist)

##3391 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab4

temp = hr_tab4.groupby('patientid')
dtime_tab4 = temp.agg(max_date=('datetime', np.max))
dtime_tab4 = dtime_tab4.reset_index()

dtime_tab4.to_csv('dtime_tab4.csv')
dtime_tab4

#max_date = discharge time/ death time for all patientids in hr_tab4

In [ ]:
#Connect to observations_table_5 - find all HR readings

hr_tab5 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_5 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab5['row_idx']=hr_tab5.index #create new index column

print(hr_tab5.shape)
hr_tab5.head()

##6,059,325 records

In [ ]:
hr_tab5_patlist = list(hr_tab5['patientid'].unique())
len(hr_tab5_patlist)

##3388 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab5

temp = hr_tab5.groupby('patientid')
dtime_tab5 = temp.agg(max_date=('datetime', np.max))
dtime_tab5 = dtime_tab5.reset_index()

dtime_tab5.to_csv('dtime_tab5.csv')
dtime_tab5

#max_date = discharge time/ death time for all patientids in hr_tab5

In [ ]:
#Connect to observations_table_6 - find all HR readings

hr_tab6 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_6 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab6['row_idx']=hr_tab6.index #create new index column

print(hr_tab6.shape)
hr_tab6.head()

##6,098,969 records

In [ ]:
hr_tab6_patlist = list(hr_tab6['patientid'].unique())
len(hr_tab6_patlist)

##3391 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab6

temp = hr_tab6.groupby('patientid')
dtime_tab6 = temp.agg(max_date=('datetime', np.max))
dtime_tab6 = dtime_tab6.reset_index()

dtime_tab6.to_csv('dtime_tab6.csv')
dtime_tab6

#max_date = discharge time/ death time for all patientids in hr_tab6

In [ ]:
#Connect to observations_table_7 - find all HR readings

hr_tab7 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_7 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab7['row_idx']=hr_tab7.index #create new index column

print(hr_tab7.shape)
hr_tab7.head()

##6,126,086 records

In [ ]:
hr_tab7_patlist = list(hr_tab7['patientid'].unique())
len(hr_tab7_patlist)

##3390 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab7

temp = hr_tab7.groupby('patientid')
dtime_tab7 = temp.agg(max_date=('datetime', np.max))
dtime_tab7 = dtime_tab7.reset_index()

dtime_tab7.to_csv('dtime_tab7.csv')
dtime_tab7

#max_date = discharge time/ death time for all patientids in hr_tab7

In [ ]:
#Connect to observations_table_8 - find all HR readings 

hr_tab8 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_8 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab8['row_idx']=hr_tab8.index #create new index column

print(hr_tab8.shape)
hr_tab8.head()

##5,964,420 records

In [ ]:
hr_tab8_patlist = list(hr_tab8['patientid'].unique())
len(hr_tab8_patlist)

##3389 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab8

temp = hr_tab8.groupby('patientid')
dtime_tab8 = temp.agg(max_date=('datetime', np.max))
dtime_tab8 = dtime_tab8.reset_index()

dtime_tab8.to_csv('dtime_tab8.csv')
dtime_tab8

#max_date = discharge time/ death time for all patientids in hr_tab8

In [ ]:
#Connect to observations_table_9 - find all HR readings

hr_tab9 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_9 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab9['row_idx']=hr_tab9.index #create new index column

print(hr_tab9.shape)
hr_tab9.head()

##6,163,893 records

In [ ]:
hr_tab9_patlist = list(hr_tab9['patientid'].unique())
len(hr_tab9_patlist)

##3389 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab9

temp = hr_tab9.groupby('patientid')
dtime_tab9 = temp.agg(max_date=('datetime', np.max))
dtime_tab9 = dtime_tab9.reset_index()

dtime_tab9.to_csv('dtime_tab9.csv')
dtime_tab9

#max_date = discharge time/ death time for all patientids in hr_tab9

In [ ]:
#Connect to observations_table_10 - find all HR readings

hr_tab10 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_10 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab10['row_idx']=hr_tab10.index #create new index column

print(hr_tab10.shape)
hr_tab10.head()

##5,770,397 records

In [ ]:
hr_tab10_patlist = list(hr_tab10['patientid'].unique())
len(hr_tab10_patlist)

##3392 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab10

temp = hr_tab10.groupby('patientid')
dtime_tab10 = temp.agg(max_date=('datetime', np.max))
dtime_tab10 = dtime_tab10.reset_index()

dtime_tab10.to_csv('dtime_tab10.csv')
dtime_tab10

#max_date = discharge time/ death time for all patientids in hr_tab10

In [ ]:
#Confirm no overlapping patientids across the 10 observations tables

pat_in_all = list(set.intersection(*map(set, [hr_tab1_patlist, hr_tab2_patlist, hr_tab3_patlist, hr_tab4_patlist, hr_tab5_patlist, hr_tab6_patlist,hr_tab7_patlist, hr_tab8_patlist, hr_tab9_patlist, hr_tab10_patlist ])))
pat_in_all

#no overlapping patientids across hr_tab1, ht_tab2, hr_tab3, hr_tab4, ht_tab5, hr_tab6, hr_tab7, ht_tab8, hr_tab9 and hr_tab10

In [ ]:
#Merge above dtime tables to create table of discharge/death times per patientid 

frames = [dtime_tab1, dtime_tab2, dtime_tab3, dtime_tab4, dtime_tab5,
         dtime_tab6, dtime_tab7, dtime_tab8, dtime_tab9, dtime_tab10]

dtimes = pd.concat(frames)

dtimes.sort_values(['patientid'], ascending=[True], inplace=True)

dtimes.to_csv('dtimes.csv')

print(dtimes.shape)
dtimes

##33,897 unique patientids (ICU admissions)

In [ ]:
#Connect to HiRID 'Patient' table (contains discharge_status info)

pat = pd.read_sql_query("SELECT * FROM hirid.patient", conn)

pat.to_csv('patient_table.csv')

print(pat.shape)
pat.head()

#33,905 records

In [ ]:
#Merge dtimes and pat dataframes

pat_info = pd.merge(pat, dtimes,  how='left', left_on=['patientid'], right_on = ['patientid'])
pat_info.rename({'max_date': 'd_time'}, axis=1, inplace=True)

pat_info.to_csv('pat_info.csv')

print(pat_info.shape)
pat_info.head()

In [ ]:
#Create ICU LOS column

pat_info['icu_los'] = pat_info['d_time'] - pat_info['admissiontime']

#convert to hours
pat_info['icu_los'] = (pat_info['icu_los'] / np.timedelta64(1, 'h'))
pat_info.rename({'icu_los': 'icu_los(hrs)'}, axis=1, inplace=True)

print(pat_info.shape)
pat_info.head()

#33,905 records 

In [ ]:
#Create fields for last hours before discharge/death (1hr/2hrs/3hrs/4hrs/7hrs/8hrs)

from datetime import timedelta

pat_info['1hr before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=1)
pat_info['2hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=2)
pat_info['3hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=3)
pat_info['4hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=4)
pat_info['5hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=5)
pat_info['6hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=6)
pat_info['7hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=7)
pat_info['8hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=8)

print(pat_info.shape)
pat_info

### 13) Explore patient characteristics of final dataset

In [ ]:
print(comp_avg_params.shape)
comp_avg_params

##2,022,310 records

In [ ]:
#Extract patientids within comp_avg_params

pat_rel = comp_avg_params['patientid'].unique()
pat_rel = pd.DataFrame(data=pat_rel)
pat_rel.columns = ['patientid']

print(pat_rel.shape)
pat_rel

In [ ]:
print(pat_info.shape)
pat_info

##33,905 unique ICU admissions

In [ ]:
#Merge pat_rel and pat_info

df_patmerge = pd.merge(pat_rel, pat_info, how='left', left_on=['patientid'], right_on=['patientid'])
print(df_patmerge.shape)
df_patmerge.head()

##20,073 records

In [ ]:
df_patmerge.describe()

In [ ]:
df_patmerge['sex'].value_counts()

In [ ]:
df_patmerge['discharge_status'].value_counts()

##17,919 discharged alive
##1,951 discharged dead

### 14) Define time periods for the last hours before discharge/death

In [ ]:
print(comp_avg_params.shape)
comp_avg_params.head()

##2,022,310 records

#### 1hr before discharge/death time

In [ ]:
#1hr before d_time

dtime_pre1hr = pat_info.copy(deep=True)
dtime_pre1hr = dtime_pre1hr[['patientid', 'd_time', '1hr before d_time']]

print(dtime_pre1hr.shape)
dtime_pre1hr

##33,905 records

In [ ]:
comp_avg_params['timepoint'] =  pd.to_datetime(comp_avg_params['timepoint'] , format='%Y-%m-%d %H:%M:%S')

comp_avg_params.dtypes

In [ ]:
#Filter for only instances with readings in the last hour before discharge/death

params_1hr = pd.merge(comp_avg_params, dtime_pre1hr, how='left', left_on=['patientid'], right_on = ['patientid'])
params_1hr = params_1hr[(params_1hr['timepoint']<params_1hr['d_time']) & (params_1hr['timepoint']>params_1hr['1hr before d_time'])]

params_1hr['Hrs before d_time'] = 1

params_1hr.to_csv("params1hr.csv")

print(params_1hr.shape)
params_1hr

##3,117 records

#### 2hrs before discharge/death time

In [ ]:
#2hrs before d_time

dtime_pre2hrs = pat_info.copy(deep=True)
dtime_pre2hrs = dtime_pre2hrs[['patientid', 'd_time', '1hr before d_time' ,'2hrs before d_time']]

print(dtime_pre2hrs.shape)
dtime_pre2hrs

In [ ]:
#Filter for only instances with readings within the 2nd hour before discharge/death

params_2hrs = pd.merge(comp_avg_params, dtime_pre2hrs, how='left', left_on=['patientid'], right_on = ['patientid'])
params_2hrs = params_2hrs[(params_2hrs['timepoint']<params_2hrs['1hr before d_time']) & (params_2hrs['timepoint']>params_2hrs['2hrs before d_time'])]

params_2hrs['Hrs before d_time'] = 2

params_2hrs.to_csv("params2hrs.csv")

print(params_2hrs.shape)
params_2hrs

##5,337 records

#### 3hrs before discharge/death time

In [ ]:
#3hrs before d_time

dtime_pre3hrs = pat_info.copy(deep=True)
dtime_pre3hrs = dtime_pre3hrs[['patientid', 'd_time', '2hrs before d_time', '3hrs before d_time']]

print(dtime_pre3hrs.shape)
dtime_pre3hrs

In [ ]:
#Filter for only instances with readings within the 3rd hour before discharge/death

params_3hrs = pd.merge(comp_avg_params, dtime_pre3hrs, how='left', left_on=['patientid'], right_on = ['patientid'])
params_3hrs = params_3hrs[(params_3hrs['timepoint']<params_3hrs['2hrs before d_time']) & (params_3hrs['timepoint']>params_3hrs['3hrs before d_time'])]

params_3hrs['Hrs before d_time'] = 3

params_3hrs.to_csv("params3hrs.csv")

print(params_3hrs.shape)
params_3hrs

##6,316 records

#### 4hrs before discharge/death time

In [ ]:
#4hrs before d_time

dtime_pre4hrs = pat_info.copy(deep=True)
dtime_pre4hrs = dtime_pre4hrs[['patientid', 'd_time', '3hrs before d_time', '4hrs before d_time']]

print(dtime_pre4hrs.shape)
dtime_pre4hrs

In [ ]:
#Filter for only instances with readings within the 4th hour before discharge/death

params_4hrs = pd.merge(comp_avg_params, dtime_pre4hrs, how='left', left_on=['patientid'], right_on = ['patientid'])
params_4hrs = params_4hrs[(params_4hrs['timepoint']<params_4hrs['3hrs before d_time']) & (params_4hrs['timepoint']>params_4hrs['4hrs before d_time'])]

params_4hrs['Hrs before d_time'] = 4

params_4hrs.to_csv("params4hrs.csv")

print(params_4hrs.shape)
params_4hrs

##7,502 records

#### 5 hrs before discharge/death time

In [ ]:
#Dataframe - 5hrs before d_time

dtime_pre5hrs = pat_info.copy(deep=True)
dtime_pre5hrs = dtime_pre5hrs[['patientid', 'd_time', '4hrs before d_time', '5hrs before d_time']]

print(dtime_pre5hrs.shape)
dtime_pre5hrs

In [ ]:
#Filter for only instances with readings within the 5th hour before discharge/death

params_5hrs = pd.merge(comp_avg_params, dtime_pre5hrs, how='left', left_on=['patientid'], right_on = ['patientid'])
params_5hrs = params_5hrs[(params_5hrs['timepoint']<params_5hrs['4hrs before d_time']) & (params_5hrs['timepoint']>params_5hrs['5hrs before d_time'])]

params_5hrs['Hrs before d_time'] = 5

params_5hrs.to_csv("params5hrs.csv")

print(params_5hrs.shape)
params_5hrs

##9,185 records

#### 6 hrs before discharge/death time

In [ ]:
#6hrs before d_time

dtime_pre6hrs = pat_info.copy(deep=True)
dtime_pre6hrs = dtime_pre6hrs[['patientid', 'd_time', '5hrs before d_time', '6hrs before d_time']]

print(dtime_pre6hrs.shape)
dtime_pre6hrs

In [ ]:
#Filter for only instances with readings within the 6th hour before discharge/death

params_6hrs = pd.merge(comp_avg_params, dtime_pre6hrs, how='left', left_on=['patientid'], right_on = ['patientid'])
params_6hrs = params_6hrs[(params_6hrs['timepoint']<params_6hrs['5hrs before d_time']) & (params_6hrs['timepoint']>params_6hrs['6hrs before d_time'])]

params_6hrs['Hrs before d_time'] = 6

params_6hrs.to_csv("params6hrs.csv")

print(params_6hrs.shape)
params_6hrs

##11,564 records

In [ ]:
#Check which patient ids have records present in: params1hr_mod, params2hrs_mod, params3hrs_mod, params4hrs_mod AND params5hrs_mod
pat1hr = set(params_1hr['patientid'])
pat2hrs = set(params_2hrs['patientid'])
pat3hrs = set(params_3hrs['patientid'])
pat4hrs = set(params_4hrs['patientid'])
pat5hrs = set(params_5hrs['patientid'])

#check common patientids
pat_comm = set(pat1hr) & set(pat2hrs) & set(pat3hrs) & set(pat4hrs) & set(pat5hrs)
print(len(pat_comm))
pat_comm

In [ ]:
params1hr_mod = params_1hr[params_1hr['patientid'].isin(pat_comm)]
params1hr_mod = params1hr_mod.drop(['rowID', '1hr before d_time'],axis=1)

params2hrs_mod = params_2hrs[params_2hrs['patientid'].isin(pat_comm)]
params2hrs_mod = params2hrs_mod.drop(['rowID', '1hr before d_time', '2hrs before d_time'],axis=1)

params3hrs_mod = params_3hrs[params_3hrs['patientid'].isin(pat_comm)]
params3hrs_mod = params3hrs_mod.drop(['rowID', '2hrs before d_time', '3hrs before d_time'],axis=1)

params4hrs_mod = params_4hrs[params_4hrs['patientid'].isin(pat_comm)]
params4hrs_mod = params4hrs_mod.drop(['rowID', '3hrs before d_time', '4hrs before d_time'],axis=1)

params5hrs_mod = params_5hrs[params_5hrs['patientid'].isin(pat_comm)]
params5hrs_mod = params5hrs_mod.drop(['rowID', '4hrs before d_time', '5hrs before d_time'],axis=1)

params6hrs_mod = params_6hrs[params_6hrs['patientid'].isin(pat_comm)]
params6hrs_mod = params6hrs_mod.drop(['rowID', '5hrs before d_time', '6hrs before d_time'],axis=1)

params7hrs_mod = params_7hrs[params_7hrs['patientid'].isin(pat_comm)]
params7hrs_mod = params7hrs_mod.drop(['rowID', '6hrs before d_time', '7hrs before d_time'],axis=1)

params8hrs_mod = params_8hrs[params_8hrs['patientid'].isin(pat_comm)]
params8hrs_mod = params8hrs_mod.drop(['rowID', '7hrs before d_time', '8hrs before d_time'],axis=1)

print(params3hrs_mod.shape)
params3hrs_mod.head()

In [ ]:
#Concatenate boave dataframes

frames = [params1hr_mod, params2hrs_mod, params3hrs_mod, params4hrs_mod, 
          params5hrs_mod]

params_full = pd.concat(frames, axis=0)

params_full = params_full.sort_values(by=['patientid','timepoint'], ascending=[True,False])
params_full = params_full.reset_index()
params_full['rowID'] = params_full.index #create unique col

params_full.to_csv('params_full.csv')

print(params_full.shape)
print('Number of patients:', params_full.patientid.nunique())
params_full.head(30)

#15,616 records

In [ ]:
#Group by patientid and Hhrs before d_time

adr_params = params_full.groupby(['patientid','Hrs before d_time'])['Adrenaline'].apply(list)
adr_params = pd.DataFrame(data=adr_params)

nor_params = params_full.groupby(['patientid','Hrs before d_time'])['Noradrenaline'].apply(list)
nor_params = pd.DataFrame(data=nor_params)

fio2_params = params_full.groupby(['patientid','Hrs before d_time'])['FiO2'].apply(list)
fio2_params = pd.DataFrame(data=fio2_params)

spo2_params = params_full.groupby(['patientid','Hrs before d_time'])['SpO2'].apply(list)
spo2_params = pd.DataFrame(data=spo2_params)

map_params = params_full.groupby(['patientid','Hrs before d_time'])['MAP'].apply(list)
map_params = pd.DataFrame(data=map_params)

hr_params = params_full.groupby(['patientid','Hrs before d_time'])['HR'].apply(list)
hr_params = pd.DataFrame(data=hr_params)

print(map_params.shape)
map_params.head()


In [ ]:
#merge above dataframes

params_mod_temp= adr_params.merge(nor_params,on=['patientid','Hrs before d_time']).merge(fio2_params,on=['patientid','Hrs before d_time']).merge(spo2_params,on=['patientid','Hrs before d_time']).merge(map_params,on=['patientid','Hrs before d_time']).merge(hr_params,on=['patientid','Hrs before d_time'])
params_mod_temp

In [ ]:
import statistics

params_mod_temp['Avg_Adrenaline'] = params_mod_temp.Adrenaline.apply(lambda x: round(statistics.mean(x),3))
params_mod_temp['Avg_Nordrenaline'] = params_mod_temp.Noradrenaline.apply(lambda x: round(statistics.mean(x),3))
params_mod_temp['Avg_FiO2'] = params_mod_temp.FiO2.apply(lambda x: round(statistics.mean(x),3))
params_mod_temp['Avg_SpO2'] = params_mod_temp.SpO2.apply(lambda x: round(statistics.mean(x),1))
params_mod_temp['Avg_MAP'] = params_mod_temp.MAP.apply(lambda x: round(statistics.mean(x),1))
params_mod_temp['Avg_HR'] = params_mod_temp.HR.apply(lambda x: round(statistics.mean(x),1))

#Drop column of lists, keep avg parameter values
cols = ['Adrenaline','Noradrenaline','FiO2','SpO2','MAP','HR']
params_mod_temp = params_mod_temp.drop(cols,axis=1)

#Rename avg_ columns
params_mod_temp.rename({'Avg_Adrenaline': 'Adrenaline'}, axis=1, inplace=True)
params_mod_temp.rename({'Avg_Nordrenaline': 'Nordrenaline'}, axis=1, inplace=True)
params_mod_temp.rename({'Avg_FiO2': 'FiO2'}, axis=1, inplace=True)
params_mod_temp.rename({'Avg_SpO2': 'SpO2'}, axis=1, inplace=True)
params_mod_temp.rename({'Avg_MAP': 'MAP'}, axis=1, inplace=True)
params_mod_temp.rename({'Avg_HR': 'HR'}, axis=1, inplace=True)
params_mod_temp = params_mod_temp.reset_index()

params_mod_temp.to_csv('params_mod_temp.csv')

print(params_mod_temp.shape)
params_mod_temp

#5,495 records

In [ ]:
pat_list = set(params_mod_temp.patientid)

#check if all ids in pat_list exist in pat table
all_pat = set(pat.patientid)

print(len(all_pat))
print(len(pat_list))

result =  all(elem in all_pat for elem in pat_list)
result

In [ ]:
#Add pat discharge_status columns

discharge_status = pat.copy(deep=True)
cols = ['admissiontime','sex','age']
discharge_status = discharge_status.drop(cols,axis=1)

params_temp_cohort = pd.merge(params_mod_temp, discharge_status, how='left', on='patientid')

#Remove all rows where discharge_status is unknown
params_temp_cohort = params_temp_cohort[params_temp_cohort['discharge_status'].notna()]

params_temp_cohort.to_csv('hirid_extval_temporal_cohort.csv')

print(params_temp_cohort.shape)
params_temp_cohort.head()

#5,320 records

In [ ]:
print(params_temp_cohort.shape)
print('Number of patients:', params_temp_cohort.patientid.nunique())
params_temp_cohort.head(15)

In [ ]:
pat_list = set(params_temp_cohort['patientid'])

pat_rel = pat[pat['patientid'].isin(pat_list)]
pat_rel.discharge_status.value_counts()

In [ ]:
params_temp_cohort = pd.read_csv('hirid_extval_temporal_cohort.csv')
params_temp_cohort = params_temp_cohort.drop('Unnamed: 0',axis=1)

print(params_temp_cohort.shape)
params_temp_cohort.head()

In [ ]:
params_temp_cohort['patientid'].nunique()